In [9]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import zipfile
import os

# Definir rutas
train_file_path = r"C:\Users\danil\Documents\GitHub2\2024-2-LAB-01-prediccion-del-default-usando-rf-DaniloSeguro\files\input\train_data.csv.zip"
test_file_path = r"C:\Users\danil\Documents\GitHub2\2024-2-LAB-01-prediccion-del-default-usando-rf-DaniloSeguro\files\input\test_data.csv.zip"

# Verificar si los archivos existen
if not os.path.exists(train_file_path):
    print(f"El archivo {train_file_path} no existe. Por favor, asegúrese de que el archivo esté en la ubicación correcta.")
if not os.path.exists(test_file_path):
    print(f"El archivo {test_file_path} no existe. Por favor, asegúrese de que el archivo esté en la ubicación correcta.")

# Función para cargar y limpiar los datos
def load_and_clean_data(file_path):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall("files/input/")
        extracted_file = zip_ref.namelist()[0]
    
    # Leer el dataset
    df = pd.read_csv(f"files/input/{extracted_file}")
    
    # Renombrar columna objetivo
    df.rename(columns={"default payment next month": "default"}, inplace=True)
    
    # Eliminar columna 'ID'
    df.drop(columns=["ID"], inplace=True)
    
    # Eliminar registros con información no disponible (NaN)
    df.dropna(inplace=True)
    
    # Agrupar valores > 4 de 'EDUCATION' en "others"
    df['EDUCATION'] = df['EDUCATION'].apply(lambda x: 4 if x > 4 else x)
    
    return df

# Cargar y limpiar datos
train_df = load_and_clean_data(train_file_path)
test_df = load_and_clean_data(test_file_path)

# Revisar datasets limpios
print("Train Data:")
print(train_df.head())
print("Test Data:")
print(test_df.head())



Train Data:
   LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0     310000    1          3         1   32      0      0      0      0   
1      10000    2          3         1   49     -1     -1     -2     -1   
2      50000    1          2         1   28     -1     -1     -1      0   
3      80000    2          3         1   52      2      2      3      3   
4     270000    1          1         2   34      1      2      0      0   

   PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0      0  ...      84373      57779      14163      8295      6000      4000   
1      2  ...       1690       1138        930         0         0      2828   
2     -1  ...      45975       1300      43987         0     46257      2200   
3      3  ...      40748      39816      40607      3700      1600      1600   
4      2  ...      22448      15490      17343         0      4000      2000   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default  
0      3000  

In [10]:
from sklearn.model_selection import train_test_split

# Separar variables explicativas (X) y objetivo (y)
X_train = train_df.drop(columns=["default"])
y_train = train_df["default"]

X_test = test_df.drop(columns=["default"])
y_test = test_df["default"]

# Verificar las dimensiones
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


X_train shape: (21000, 23), y_train shape: (21000,)
X_test shape: (9000, 23), y_test shape: (9000,)


In [11]:
# Importar librerías necesarias
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

# Identificar columnas categóricas y numéricas
categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']  # Variables categóricas
numerical_features = [col for col in X_train.columns if col not in categorical_features]

# Crear transformador para codificación One-Hot de variables categóricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Crear el preprocesador usando ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # Mantener variables numéricas sin cambios
)

# Crear el pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Entrenar el modelo en el conjunto de entrenamiento
pipeline.fit(X_train, y_train)

# Imprimir mensaje de éxito
print("Pipeline creado y modelo entrenado correctamente.")


Pipeline creado y modelo entrenado correctamente.


In [12]:
# Importar librerías necesarias
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, balanced_accuracy_score

# Definir los hiperparámetros a optimizar
param_grid = {
    'classifier__n_estimators': [100, 200, 300],   # Número de árboles en el bosque
    'classifier__max_depth': [None, 10, 20],       # Profundidad máxima de los árboles
    'classifier__min_samples_split': [2, 5, 10],   # Mínimo número de muestras para dividir un nodo
    'classifier__min_samples_leaf': [1, 2, 4],     # Mínimo número de muestras en una hoja
}

# Crear un scorer para la precisión balanceada
scorer = make_scorer(balanced_accuracy_score)

# Configurar la validación cruzada
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=10,                          # Número de folds para validación cruzada
    scoring=scorer,                 # Métrica de precisión balanceada
    n_jobs=-1,                      # Usar todos los núcleos disponibles
    verbose=2                       # Mostrar el progreso
)

# Ejecutar la optimización de hiperparámetros
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Asignar el mejor modelo a una variable
best_model = grid_search.best_estimator_hgb

# Imprimir el rendimiento en el conjunto de prueba
from sklearn.metrics import accuracy_score

y_pred_test = best_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Precisión del modelo optimizado en el conjunto de prueba: {test_accuracy:.4f}")


Fitting 10 folds for each of 81 candidates, totalling 810 fits
Mejores hiperparámetros encontrados:
{'classifier__max_depth': None, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 300}
Precisión del modelo optimizado en el conjunto de prueba: 0.8292


In [14]:
import joblib
import gzip
import os

# Definir la ruta correcta usando os.path.join
output_dir = "files/models"
os.makedirs(output_dir, exist_ok=True)  # Crear el directorio si no existe
model_path = os.path.join(output_dir, "model.pkl.gz")

# Asegurar que las rutas tengan slashes correctos
model_path = model_path.replace("\\", "/")  # Reemplazar backslashes con slashes

# Guardar el modelo comprimido usando gzip
with gzip.open(model_path, 'wb') as f:
    joblib.dump(best_model, f)

print(f"Modelo guardado correctamente en: {model_path}")



Modelo guardado correctamente en: files/models/model.pkl.gz


In [19]:
# Importar librerías necesarias
from sklearn.metrics import precision_score, balanced_accuracy_score, recall_score, f1_score
import json
import os

# Crear una función para calcular las métricas
def calculate_metrics(model, X, y, dataset_name):
    y_pred = model.predict(X)
    metrics = {
        'dataset': dataset_name,
        'precision': precision_score(y, y_pred),
        'balanced_accuracy': balanced_accuracy_score(y, y_pred),
        'recall': recall_score(y, y_pred),
        'f1_score': f1_score(y, y_pred)
    }
    return metrics

# Calcular métricas para entrenamiento y prueba
train_metrics = calculate_metrics(best_model, X_train, y_train, 'train')
test_metrics = calculate_metrics(best_model, X_test, y_test, 'test')

# Crear directorio de salida si no existe
output_dir = "files/output"
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
metrics_path = os.path.join(output_dir, "metrics.json")

# mostrar las metricas
print("Métricas de entrenamiento:")
print(json.dumps(train_metrics, indent=2))
print("\nMétricas de prueba:")
print(json.dumps(test_metrics, indent=2))




Métricas de entrenamiento:
{
  "dataset": "train",
  "precision": 0.9581774906494389,
  "balanced_accuracy": 0.7942956226666198,
  "recall": 0.5961497778718003,
  "f1_score": 0.7350026082420449
}

Métricas de prueba:
{
  "dataset": "test",
  "precision": 0.6600688468158348,
  "balanced_accuracy": 0.6730383115731366,
  "recall": 0.4017810371922472,
  "f1_score": 0.4995115597525236
}


In [17]:
# Importar librerías necesarias
from sklearn.metrics import confusion_matrix

# Función para calcular la matriz de confusión y formatearla
def get_confusion_matrix(model, X, y, dataset_name):
    y_pred = model.predict(X)
    cm = confusion_matrix(y, y_pred)
    
    cm_dict = {
        "type": "cm_matrix",
        "dataset": dataset_name,
        "true_0": {"predicted_0": int(cm[0, 0]), "predicted_1": int(cm[0, 1])},
        "true_1": {"predicted_0": int(cm[1, 0]), "predicted_1": int(cm[1, 1])}
    }
    return cm_dict

# Calcular matrices de confusión para entrenamiento y prueba
train_cm = get_confusion_matrix(best_model, X_train, y_train, "train")
test_cm = get_confusion_matrix(best_model, X_test, y_test, "test")

# Cargar las métricas existentes o crear una lista vacía si el archivo no existe
if os.path.exists(metrics_path):
    with open(metrics_path, 'r') as f:
        metrics = json.load(f)
else:
    metrics = []

# Actualizar las métricas con las matrices de confusión
metrics.append(train_cm)
metrics.append(test_cm)

# Guardar las métricas actualizadas en el archivo JSON
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=4)

# Mostrar las matrices de confusión
print("Matrices de confusión calculadas y añadidas:")
print(json.dumps([train_cm, test_cm], indent=4))


Matrices de confusión calculadas y añadidas:
[
    {
        "type": "cm_matrix",
        "dataset": "train",
        "true_0": {
            "predicted_0": 16150,
            "predicted_1": 123
        },
        "true_1": {
            "predicted_0": 1909,
            "predicted_1": 2818
        }
    },
    {
        "type": "cm_matrix",
        "dataset": "test",
        "true_0": {
            "predicted_0": 6696,
            "predicted_1": 395
        },
        "true_1": {
            "predicted_0": 1142,
            "predicted_1": 767
        }
    }
]
